In [1]:
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

gcv = GridSearchCV(SVC(gamma="auto"), {
    "C":[1,10,20],
    "kernel":["rbf","linear"]
}, cv=5, return_train_score=False)

gcv.fit(iris.data, iris.target) # High computational cost hence not very used in practicle life
gcv.cv_results_

{'mean_fit_time': array([0.00180478, 0.00080056, 0.00079856, 0.00059757, 0.00060315,
        0.00039825]),
 'std_fit_time': array([0.00159709, 0.00040029, 0.00039941, 0.00048796, 0.0004925 ,
        0.00048776]),
 'mean_score_time': array([0.00039892, 0.00039716, 0.00099983, 0.00040097, 0.0004014 ,
        0.        ]),
 'std_score_time': array([4.88581565e-04, 4.86416964e-04, 1.19728863e-05, 4.91093068e-04,
        4.91625645e-04, 0.00000000e+00]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [4]:
df = pd.DataFrame(gcv.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001805,0.001597,0.000399,0.000489,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000801,0.000400,0.000397,0.000486,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000799,0.000399,0.001000,0.000012,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000598,0.000488,0.000401,0.000491,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000603,0.000492,0.000401,0.000492,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5


In [5]:
gcv.best_params_

{'C': 1, 'kernel': 'rbf'}

In [9]:
from sklearn.model_selection import RandomizedSearchCV

rcv = RandomizedSearchCV(SVC(gamma="auto"), {
    "C":[1,10,20],
    "kernel":["rbf","linear"]
}, cv=5, return_train_score=False, n_iter=2)

rcv.fit(iris.data, iris.target)
pd.DataFrame(rcv.cv_results_)[["param_C", "param_kernel", "mean_test_score"]] # work in practicle life

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,1,linear,0.980000


In [10]:
rcv.best_params_

{'kernel': 'linear', 'C': 1}

In [11]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [17]:
model_param = {
    "svm" : {
        "model": SVC(),
        "params" : {
            "C" : [1,10,20],
            "kernel": ["rbf", "linear"]
        }
    },
    "random_forest" : {
        "model": RandomForestClassifier(),
        "params" : {
            "n_estimators":[1, 5, 10]
        }
    },
    "logistic_regression" : {
        "model": LogisticRegression(multi_class="auto", solver="liblinear"),
        "params" : {
            "C" : [1, 5, 10]
        }
    }
}

In [18]:
scores = []

for model_name, mp in model_param.items():
    rcv = RandomizedSearchCV(mp["model"], mp["params"], cv=5, return_train_score=False, n_iter=2)
    rcv.fit(iris.data, iris.target)
    scores.append({
        "model_name":model_name,
        "best_param":rcv.best_params_,
        "best_score":rcv.best_score_
    })

scores

[{'model_name': 'svm',
  'best_param': {'kernel': 'linear', 'C': 10},
  'best_score': 0.9733333333333334},
 {'model_name': 'random_forest',
  'best_param': {'n_estimators': 10},
  'best_score': 0.9466666666666667},
 {'model_name': 'logistic_regression',
  'best_param': {'C': 10},
  'best_score': 0.9666666666666668}]

In [19]:
pd.DataFrame(scores, columns=["model_name", "best_param", "best_score"])

,model_name,best_param,best_score
0,svm,"{'kernel': 'linear', 'C': 10}",0.973333
1,random_forest,{'n_estimators': 10},0.946667
2,logistic_regression,{'C': 10},0.966667
